In [13]:
import numpy as np
import glob
import os
from skimage.io import imread, imsave
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [16]:
def data_augmentation(image_path, image_subindex, output_directory, augmentation_count=10):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    img = load_img(image_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    # Define directory based on class_name extracted from image_subindex or any logic you follow
    class_name = image_subindex.split('_')[0]  # Customize this based on your naming convention
    save_dir = f"{output_directory}/{class_name}"
    os.makedirs(save_dir, exist_ok=True)

    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=save_dir, save_prefix=f"aug_{image_subindex}", save_format="jpg"):
        i += 1
        if i >= augmentation_count:
            break

def crop_and_augment_images(data_directory, output_directory):
    txt_files = glob.glob(f"{data_directory}/*.txt")
    for txt_file in txt_files:
        base_name = os.path.splitext(os.path.basename(txt_file))[0]
        image_path = f"{data_directory}/{base_name}.jpg"
        if not os.path.exists(image_path):
            continue  # Skip if the corresponding image does not exist

        # Crop images based on coordinates and labels from txt file
        save_crop_image(txt_file, image_path, base_name, output_directory)
        
        # Augment the cropped images
        data_augmentation(image_path, base_name, output_directory)

def save_crop_image(path_txt, path_image, image_subindex, output_directory):
    with open(path_txt, 'r') as file:
        lines = file.readlines()[2:52]  # Assuming relevant data starts from line 3
    
    image = imread(path_image)
    for n, line in enumerate(lines):
        parts = line.split(",")
        x, y = map(float, parts[:2])  # Convert to float for coordinates
        label = int(parts[2])  # Assuming label is the third part and is an integer
        # Adjust cropping logic as needed, considering x and y as floats
        x, y = int(x), int(y)  # Convert coordinates to integers if necessary for indexing
        cropped_image = image[max(0, y-15):y+15, max(0, x-15):x+15]

        label_dirs = ["Coral", "DCP", "Rock", "Red algae", "Green algae", "Others"]
        label_dir = label_dirs[label] if label < len(label_dirs) else "Others"
        save_path = f"{output_directory}/{label_dir}/{label_dir}-{image_subindex}-{n}.jpg"
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        imsave(save_path, cropped_image)


In [17]:
if __name__ == "__main__":
    data_directory = "/Volumes/Shaq Toshiba/Cuba CPCe Images/201705251_ROV 17-15_421"
    output_directory = "/Users/shagundeepsingh/Documents/coral/preprocessing"
    crop_and_augment_images(data_directory, output_directory)

/var/folders/b8/_x62wfl112g_8f32jdl3v95h0000gn/T/ipykernel_21780/705642084.py:57: UserWarning: /Users/shagundeepsingh/Documents/coral/preprocessing/Red algae/Red algae-201705251_T-10-52-09_118-7.jpg is a low contrast image
  imsave(save_path, cropped_image)
/var/folders/b8/_x62wfl112g_8f32jdl3v95h0000gn/T/ipykernel_21780/705642084.py:57: UserWarning: /Users/shagundeepsingh/Documents/coral/preprocessing/Others/Others-201705251_T-10-50-31_112-16.jpg is a low contrast image
  imsave(save_path, cropped_image)
/var/folders/b8/_x62wfl112g_8f32jdl3v95h0000gn/T/ipykernel_21780/705642084.py:57: UserWarning: /Users/shagundeepsingh/Documents/coral/preprocessing/Others/Others-201705251_T-10-50-31_112-17.jpg is a low contrast image
  imsave(save_path, cropped_image)
/var/folders/b8/_x62wfl112g_8f32jdl3v95h0000gn/T/ipykernel_21780/705642084.py:57: UserWarning: /Users/shagundeepsingh/Documents/coral/preprocessing/Red algae/Red algae-201705251_T-10-50-31_112-30.jpg is a low contrast image
  imsave(sav